### The goal of this portion is to build a prototype of the unsupervised portion of the Hybrid model by using the Surprise package

In [ ]:
import numpy as np
import pandas as pd
import math
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy
import surprise
%matplotlib inline

In [4]:
#%conda install -c conda-forge scikit-surprise

In [5]:
testing_df=pd.read_pickle("./testing.pkl")
testing_df.drop('Unnamed: 0',axis=1,inplace=True)


In [6]:
frequency_df=pd.DataFrame(testing_df.groupby(['userID']).size()).reset_index()
frequency_df.rename(columns={0:'frequency'},inplace=True)
testing_new_df=testing_df.merge(frequency_df,how='right',left_on='userID',right_on='userID')

In [7]:
testing_new_df.drop_duplicates(subset=['userID'],keep='last',inplace=True)
testing_new_df.reset_index(inplace=True)

In [8]:
user_item_df=testing_new_df.loc[testing_new_df['frequency']>=10,('userID','asin','rating')].reset_index().drop(columns=['index'])

In [9]:
user_item_surprise=user_item_df.copy()

user_item_surprise.columns=['user','item','rating']

In [10]:
user_item_surprise['user']=user_item_surprise.index
user_item_surprise['item']=user_item_surprise.index

In [11]:
user_item_surprise.head()

,user,item,rating
0,0,0,4.0
1,1,1,4.0
2,2,2,3.0
3,3,3,5.0
4,4,4,4.0


In [12]:
user_item_matrix=pd.pivot_table(user_item_surprise,index = 'user', columns ='item', values = 'rating').fillna(0)

In [13]:
reader = surprise.reader.Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(user_item_surprise[['user','item','rating']],reader)

In [14]:
algo = SVD(verbose=True)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0867  1.0675  1.0943  1.1077  1.0998  1.0912  0.0137  
MAE (testset)     0.8454  0.8409  0.8481  0.8598  0.8512  0.8491  0.0063  
Fit time          0.84    0.80    0.80    0.88    0.83    0.83    0.03    
Test time         0.02    0.03    0.02    0.03    0.02    0.02    0.01    


{'test_rmse': array([1.08671949, 1.06745781, 1.09434843, 1.10769227, 1.09976932]),
 'test_mae': array([0.84544503, 0.84094447, 0.84812504, 0.85976942, 0.85119335]),
 'fit_time': (0.8375344276428223,
  0.7986569404602051,
  0.8036494255065918,
  0.8846559524536133,
  0.8277406692504883),
 'test_time': (0.018527746200561523,
  0.033478736877441406,
  0.016397714614868164,
  0.02737712860107422,
  0.017697572708129883)}

In [15]:
trainset, testset = train_test_split(data, test_size=.25)

In [16]:
algo.fit(trainset)
predictions = algo.test(testset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [17]:
pred = algo.predict(1, 6660, verbose=True)

user: 1          item: 6660       r_ui = None   est = 4.37   {'was_impossible': False}


In [176]:
float(pred.est)

4.445547939508856

In [208]:
def hybrid(userid,df,unsupervised_predict):
    recommend_list=[]
    i=0
    while i<len(df):
        pred=algo.predict(userid, i, verbose=False)
        if float(pred.est) >= 4.6:
            recommend_list.append(df['asin'][i])
        i+=1
    return len(recommend_list)
    
    